In [1]:
import os
os.chdir('..')

In [2]:
from sim_circuit import QuantumRegister, QuantumCircuit
from util import all_close

def geometric_sequence_circuit(n, theta):

    N = 2**n

    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for j in range(n): # <1>
        qc.h(q[j])
        qc.p(2 ** j * theta, q[j])

    assert all_close(qc.run(), [1/sqrt(N)*cis(k*theta) for k in range(N)]) # <2>

In [3]:
from math import pi

def encode_frequency(n, v):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for j in range(n):
        qc.h(q[j])
        qc.p(2 * pi / 2 ** (n - j) * v, q[j]) # <1>

    qc.report('geometric_sequence')

    qc.iqft(range(n))

    qc.report('iqft')

    return qc

In [4]:
from util import print_state_table

n = 3
v = 4.7

state = encode_frequency(n, v).run()

In [5]:
print_state_table(state)



Outcome   Binary        Amplitude        Magnitude   Direction      Amplitude Bar         Probability
----------------------------------------------------------------------------------------------------
  0        000     +0.0986 + 0.0364*i     0.1051     20.26         ##                       0.011
  1        001     +0.0748 + 0.0691*i     0.1018     42.73         ##                       0.0104
  2        010     +0.0485 + 0.1053*i     0.1159     65.27         ##                       0.0134
  3        011     +0.0064 + 0.1632*i     0.1633     87.75         ###                      0.0267
  4        100     -0.1289 + 0.3495*i     0.3725     110.24        #######                  0.1388
  5        101     +0.5840 - 0.6318*i     0.8604     -47.25        #################        0.7402
  6        110     +0.1880 - 0.0866*i     0.2070     -24.73        ####                     0.0428
  7        111     +0.1287 - 0.0051*i     0.1288     -2.27         ##                       0.0166


In [6]:
def complex_sincd(n, v):
    N = 2**n
    return [prod(cos((v-k)*pi/2**(m+1)) for m in range(n))*cis((N-1)/N * (v-k)*pi) for k in range(2**n)]

In [7]:
def complex_sincd_combined_cis(n, v):
    N = 2**n
    return [prod(cos((v-k)*pi/2**(m+1)) for m in range(n))*cis((N-1)/N * (v-k)*pi) for k in range(2**n)]

In [8]:
def complex_sincd_sum(n, v):
    # same as value encoding state
    N = 2**n
    return [2/N*sum(cos((2*m+1)*(v-k)*pi/N) for m in range(N//2))*cis((N-1) * (v-k)*pi/N) for k in range(N)]


def test_sum_form():
    n = 3
    v = 4.7
    assert all_close(complex_sincd(n, v), complex_sincd_sum(n, v))

In [9]:
def recursive_complex_discrete_sinc(n, v):
    if n == 1:
        return [cis(v*pi/2)*cos(v*pi/2), -1j*cis(v*pi/2)*sin(v*pi/2)]

    a = recursive_complex_discrete_sinc(n-1, v)

    return ([a[k]*cis((v - k)*pi/2**n)*cos((v - k)*pi/2**n) for k in range(2**(n-1))] +
            [a[k]*cis((v - k)*pi/2**n)*-1j*sin((v - k)*pi/2**n) for k in range(2**(n-1))])

In [10]:
def geom_alt(n, v):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for j in range(n):
        qc.h(q[j])
        qc.p(pi * 2 ** -j * v, q[j])

    return qc

In [11]:
def encode_frequency_q_alt(n, v):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for j in range(n):
        qc.h(q[j])
        qc.p(pi * 2 ** -j * v, q[j])

    qc.report('signal')

    qc.iqft(range(n)[::-1], swap=False)

    qc.report('iqft')

    return qc

In [12]:
from util import is_close

def test_discrete_sinc_by_digit():
    n = 3
    v = 4.7

    for l in range(2**n): # <1>
        s = bin(l)[2:].zfill(n)
        assert(len(s) == n)
        p = 1
        k = 0
        for m in range(n): # <2>
            if s[m] == '0':
                p *= cos((v - k)*pi/2**(m+1))**2
            else:
                p *= sin((v - k)*pi/2**(m+1))**2
                k += 2**m

        assert isclose(p, prod(cos((v-k)*pi/2**(j+1)) for j in range(n))**2) # <3>

In [13]:
def recursive_discrete_sinc(n, v):
    if n == 1:
        return [cos(v*pi/2)**2, sin(v*pi/2)**2]

    p = recursive_discrete_sinc(n-1, v)

    return [p[k]*cos((v - k)*pi/2**n)**2 for k in range(2**(n-1))] + [p[k]*sin((v - k)*pi/2**n)**2 for k in range(2**(n-1))]

In [14]:
def discrete_sinc_coin_flips(n, v, count=10000):
    samples = []
    for _ in range(count):
        k = 0
        for m in range(n):
            flip = np.random.binomial(1, sin((v-k)*pi/2**(m+1))**2)
            k += flip*2**m

        samples.append(k)

    return samples

In [15]:
def raised_cosine(n, mu):
    N = 2 ** n
    assert (0 <= mu < 2 ** n)

    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    qc.h(q[n - 1])
    qc.p(-pi * mu / N * 2, q[n - 1])

    qc.report('fourier_coefficients')

    qc.qft(q[::-1], False) # <1>

    qc.report('qft')

    return qc

In [16]:
qc = raised_cosine(3, 3.25)
state = qc.run()

In [17]:
print_state_table(state)



Outcome   Binary        Amplitude        Magnitude   Direction      Amplitude Bar         Probability
----------------------------------------------------------------------------------------------------
  0        000     +0.0421 - 0.1389*i     0.1451     -73.14        ##                       0.0211
  1        001     +0.2012 - 0.2452*i     0.3172     -50.63        ######                   0.1006
  2        010     +0.3889 - 0.2079*i     0.4410     -28.13        ########                 0.1945
  3        011     +0.4952 - 0.0488*i     0.4976     -5.63         #########                0.2476
  4        100     +0.4579 + 0.1389*i     0.4785     16.87         #########                0.229
  5        101     +0.2988 + 0.2452*i     0.3865     39.37         #######                  0.1494
  6        110     +0.1111 + 0.2079*i     0.2357     61.88         ####                     0.0556
  7        111     +0.0048 + 0.0488*i     0.0490     84.38                                  0.0024


In [18]:
from math import sqrt, cos
from util import cis

N = 8
mu = 3.25
a = [sqrt(2/N) * cos((k - mu)*pi/N) * cis((k-mu)*pi/N) for k in range(N)]
assert all_close(state, a)

In [19]:
s = N / 2
p = [1 / (2 * s) * (1 + cos((x - mu) / s * pi)) for x in range(N)]
p1 = [1 / s * cos((x - mu) / (2 * s) * pi) ** 2 for x in range(N)]

probs = [2/N*(cos((k - mu)*pi/N))**2 for k in range(N)] 

assert all_close(p, probs)
assert all_close(p1, probs)

In [20]:
from math import acos

def sin_4(n):
    theta = acos(sqrt(2 / 3))
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    qc.ry(2 * theta, q[n - 1])
    qc.p(pi, q[n - 1])
    qc.cry(pi / 2, q[n - 1], q[0])

    for i in range(1, n - 1):
        qc.cx(q[0], q[i])

    qc.report('frequencies')

    qc.qft(q[::-1], False)

    qc.report('qft')

    return qc

In [21]:
n = 3           
N = 2 ** n      
qc = sin_4(n)   
state = qc.run()

In [22]:
print_state_table(state)



Outcome   Binary        Amplitude        Magnitude   Direction      Amplitude Bar         Probability
----------------------------------------------------------------------------------------------------
  0        000     +0.0000 + 0.0000*i     0.0000                                            0.0
  1        001     +0.0846 + 0.0000*i     0.0846     0.0           #                        0.0072
  2        010     +0.2887 + 0.0000*i     0.2887     0.0           #####                    0.0833
  3        011     +0.4928 + 0.0000*i     0.4928     0.0           #########                0.2429
  4        100     +0.5774 + 0.0000*i     0.5774     0.0           ###########              0.3334
  5        101     +0.4928 + 0.0000*i     0.4928     0.0           #########                0.2429
  6        110     +0.2887 + 0.0000*i     0.2887     0.0           #####                    0.0833
  7        111     +0.0846 + 0.0000*i     0.0846     0.0           #                        0.0072


In [23]:
from math import sin

s = [sqrt(8 / (3 * N)) * (sin(k * pi / N)) ** 2 for k in range(N)]
assert all_close(state, s)
p = [8 / 3 / N * (sin(k * pi / N)) ** 4 for k in range(N)]  # alternatively: 8/3/N*(cos((k - N/2)*pi/N))**4
assert all_close([abs(state[k])**2 for k in range(N)], p)

In [24]:
theta = -pi/2         
q = QuantumRegister(n)
qc = QuantumCircuit(q)
qc.ry(theta, q[0])    
qc.report('state')

([1, 0, 0, 0, 0, 0, 0, 0],
 [0.7071067811865476, -0.7071067811865475, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [25]:
s = [cos(k*pi/2 - theta/2) if k in [0, 1] else 0 for k in range(N)]
assert all_close(qc.reports['state'][2], s)

In [26]:
qc.qft(q)       
qc.report('qft')

([0.7071067811865476, -0.7071067811865475, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [2.7755575615628914e-17,
  (0.0732233047033631-0.1767766952966368j),
  (0.24999999999999992-0.24999999999999992j),
  (0.42677669529663675-0.17677669529663684j),
  (0.4999999999999999+0j),
  (0.42677669529663675+0.1767766952966368j),
  (0.24999999999999997+0.24999999999999992j),
  (0.07322330470336316+0.17677669529663684j)])

In [27]:
s = [1/sqrt(N)*cos(theta/2) + 1/sqrt(N)*sin(theta/2)*cis(k*2*pi/N) for k in range(N)]
assert all_close(qc.reports['qft'][2], s)

In [28]:
def one_over_sine(n):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for i in range(1, n):
        qc.h(i)

    qc.x(0)
    qc.iqft(q[::-1], swap=False)

    return qc

In [29]:
qc = one_over_sine(4)
state = qc.run()

In [30]:
print_state_table(state)



Outcome   Binary        Amplitude        Magnitude   Direction      Amplitude Bar         Probability
----------------------------------------------------------------------------------------------------
   0       0000     +0.7071 + 0.0000*i     0.7071     0.0           ##############           0.5
   1       0001     -0.0884 + 0.4444*i     0.4531     101.25        #########                0.2053
   2       0010     +0.0000 + 0.0000*i     0.0000                                            0.0
   3       0011     -0.0884 + 0.1323*i     0.1591     123.75        ###                      0.0253
   4       0100     +0.0000 + 0.0000*i     0.0000                                            0.0
   5       0101     -0.0884 + 0.0591*i     0.1063     146.24        ##                       0.0113
   6       0110     +0.0000 + 0.0000*i     0.0000                                            0.0
   7       0111     -0.0884 + 0.0176*i     0.0901     168.74        #                        0.0081
   8   

In [31]:
from math import tan
n = 4
N = 2**n

qc = one_over_sine(n)
state = qc.run()

assert is_close(abs(state[0])**2, 1/2)

for k in range(1, N):
    if k%2 == 1:
        assert is_close(abs(state[k])**2, (sqrt(2)/N/sin(k*pi/N))**2)
        assert all_close([state[k]], [sqrt(2)/N*(-1 + 1j*1/tan(k*pi/N))])
    else:
        assert is_close(abs(state[k])**2, 0)